##### 线性回归的手动实现

In [15]:
# 加载模块
import random

# 加载绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# 加载计算模块
import numpy as np

# 加载深度神经网络算法的核心数据结构与计算模块
import torch
# =====> 第一部分: 数据
from torch.utils.data import Dataset, TensorDataset, DataLoader
# =====> 第二部分: 搭建深度神经网络模型
import torch.nn as nn  # 深度神经网络模型中的各种层(内置整合信息的方式)
from torch.nn import functional as F  # 深度神经网络模型中的各种基于张量的整合信息和加工信息的方式
# =====> 第三部分: 训练深度神经网络模型
import torch.optim as optim
# =====> 第四部分: 辅助工具
from tensorboardX import SummaryWriter

# 加载自定义模块
from torchLearning import *

In [16]:
tensorGenReg?

Signature:
tensorGenReg(
    num_samples=1000,
    w=[2, -1, 1],
    bias=True,
    delta=0.01,
    degree=1,
)
Docstring:
回归类任务数据集创建函数:
@param num_samples: 数据集中的样本个数
@param w: 数据集中特征变量前的权重, 包含截距项
@param bias: 是否包含截距项
@param delta: 控制噪声的大小
@param degree: 每一个特征变量的系数

@return: 特征张量X和标签张量y组成的元组
File:      ~/Desktop/prep_PhD/DL/codes/torchLearning.py
Type:      function

In [17]:
torch.manual_seed(55)
X, y = tensorGenReg()

In [20]:
# 查看特征张量的维度和形状
# X.ndim
# X.shape

In [23]:
# 查看标签的维度和形状
# y.ndim
# y.shape

##### 自定义神经网络模型: 直接基于张量的计算完成深度神经网络建模

In [24]:
def linear_regression(X, w):
    return torch.mm(X, w)

In [25]:
def squared_loss(y_hat, y):
    # 计算数据集中样本的个数
    num_ = y.numel()
    # 计算SSE
    sse = torch.sum((y_hat.reshape(-1, 1) - y.reshape(-1, 1)) ** 2)
    return sse / num_

In [63]:
def sgd(params, lr):
    params.data = params.data - lr * params.grad
    params.grad.zero_()

In [26]:
w = torch.tensor(2., requires_grad=True, dtype=torch.float32)
w

tensor(2., requires_grad=True)

In [27]:
w.is_leaf

True

In [28]:
w1 = w * 2
w1

tensor(4., grad_fn=<MulBackward0>)

In [29]:
w1.is_leaf

False

In [30]:
# 在Pytorch进行计算的时候, 可以将一个设置了requires_grad=True的张量运算的结果存储在计算机的内存中
# 并且给该内存区域起一个其它不同于当前张量的别名
w = torch.tensor(2., requires_grad=True, dtype=torch.float32)
w1 = w * 2
w1

tensor(4., grad_fn=<MulBackward0>)

In [34]:
# 但是不可以将计算的结果存储在计算机的内存空间中, 然后给该内存区域起一个当前张量的别名
w = torch.tensor(2., requires_grad=True, dtype=torch.float32)
w -= w * 2
w

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [35]:
w = torch.tensor(2., requires_grad=True, dtype=torch.float32)
w = w * 2
w

tensor(4., grad_fn=<MulBackward0>)

In [36]:
w.is_leaf

False

In [37]:
w = torch.tensor(2., requires_grad=True, dtype=torch.float32)
w = w * 2

In [38]:
w.backward()

In [39]:
w.grad

/var/folders/ht/7tjg01r945gch9nd9n7s7c3c0000gn/T/ipykernel_10257/447174752.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:494.)
  w.grad


In [40]:
w = torch.tensor(2., requires_grad=True, dtype=torch.float32)
w

tensor(2., requires_grad=True)

In [41]:
with torch.no_grad():
    w -= w * 2
w

tensor(-2., requires_grad=True)

In [43]:
w.is_leaf

True

In [44]:
w = torch.tensor(2., requires_grad=True, dtype=torch.float32)
w

tensor(2., requires_grad=True)

In [46]:
w.detach_()  # 从计算图中移除了该叶子节点

tensor(2.)

In [47]:
w.is_leaf

True

In [48]:
w -= w * 2
w

tensor(-2.)

In [49]:
w.requires_grad = True

In [50]:
w

tensor(-2., requires_grad=True)

In [51]:
w.is_leaf

True

In [52]:
w = torch.tensor(2., requires_grad=True, dtype=torch.float32)
w

tensor(2., requires_grad=True)

In [53]:
w.data

tensor(2.)

In [54]:
w

tensor(2., requires_grad=True)

In [55]:
w.data -= w * 2

In [56]:
w

tensor(-2., requires_grad=True)

In [57]:
w.data

tensor(-2.)

In [58]:
w.is_leaf

True

In [74]:
torch.manual_seed(55)

# 加载数据
X, y = tensorGenReg()

# 将完整的数据集按照一定的大小划分为若干个互不相交的子数据集
batch_size = 10
batched_dataset = split_dataset(batch_size=batch_size, X=X, y=y)
# 设置将完整的数据集学习的次数
n_epochs = 3
# 设置优化器的参数
eta = 0.03
# 随机初始化模型的参数
w = torch.zeros(size=(X.shape[1], 1), requires_grad=True, dtype=torch.float32)

In [59]:
split_dataset?

Signature: split_dataset(batch_size, X, y)
Docstring:
将完整的数据集(特征张量, 标签)按照相同的大小, 划分为若干个互不相交的子集
@param batch_size: 每一个子数据集中样本的个数
@param X: 包含所有样本的特征张量
@param y: 包含所有样本的标签

@return batched_dataset: 列表, 包含有一定数量的元组, 每一个元组的第一个元素是子集的特征张量, 第二个元素是子集的标签
File:      ~/Desktop/prep_PhD/DL/codes/torchLearning.py
Type:      function

In [67]:
net = linear_regression
loss = squared_loss

In [68]:
type(squared_loss)

function

In [75]:
# 训练深度神经网络模型
# 1.将完整的数据集学习多遍
for i_epoch in range(n_epochs):
    # 2.在每一遍完整学习数据集的时候, 将数据集拆分为若干个互不相交的子集进行学习
    for i_batch in batched_dataset:
        i_X = i_batch[0]
        i_y = i_batch[1]
        # 前向传播
        y_hat = net(i_X, w)
        # 计算损失
        l = squared_loss(y_hat, i_y)
        # 反向传播
        l.backward()
        # 使用优化器更新模型的参数
        sgd(w, eta)

    # 完整学习完一遍数据集之后, 计算当前模型的参数在完整的数据集上的损失
    # 前向传播
    y_hat = net(X, w)
    # 计算损失
    l = squared_loss(y_hat, y)
    print("epoch %d, loss %f" % (i_epoch+1, l))

epoch 1, loss 0.000119
epoch 2, loss 0.000095
epoch 3, loss 0.000095


In [76]:
w

tensor([[ 2.0007],
        [-0.9996],
        [ 0.9996]], requires_grad=True)

In [77]:
# 使用tensorboardX模块来记录训练深度神经网络模型过程中核心指标的变化
writer = SummaryWriter(logdir="./reg_loss")

In [79]:
# 加载数据
torch.manual_seed(55)
X, y = tensorGenReg()

# 搭建深度神经网络模型
net = linear_regression
w = torch.zeros(size=(X.shape[1], 1), requires_grad=True)

# 训练深度神经网络模型
batch_size = 10
eta = 0.03
# =====> 将完整的数据集训练多遍
n_epochs = 3
for i_epoch in range(n_epochs):
    # =====> 将一个完整的数据集划分为若干个互不相交的子数据集进行训练
    for i_batch in split_dataset(batch_size=batch_size, X=X, y=y):
        # 获取当前的子数据集中的特征张量和标签
        i_X, i_y = i_batch[0], i_batch[1]
        # 1.前向传播
        y_hat = net(i_X, w)
        # 2.计算损失
        loss = squared_loss(y_hat, i_y)
        # 3.反向传播
        loss.backward()
        # 4.优化器利用反向传播的信息更新模型的参数
        sgd(params=w, lr=eta)
    # 将一个完整的数据集学习完一遍后, 计算以当前模型的参数学习完整的数据集的损失函数值
    loss = squared_loss(net(X, w), y)
    # 使用工具记录关键信息
    writer.add_scalar("mul", loss, i_epoch)

##### =====> 数据

In [85]:
torch.manual_seed(55)

# 加载数据集
X, y = tensorGenReg()
X = X[:, :-1]
# 使用TensorDataset类, 将特征张量和标签打包存储在元组中
dataset = TensorDataset(X, y)
# 设置将一个完整的数据集划分为若干个互不相交的子集的子集大小
batch_size = 10
# 将一个完整的数据集按照固定的子集大小划分为若干个子集
batched_dataset = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last=False)

##### =====> 搭建深度神经网络模型

In [86]:
class GeoffNet(nn.Module):
    # 构造器 + 类/对象属性
    # => 子类的对象调用父类的构造器
    # => 搭建深度神经网络模型的架构
    def __init__(self, in_features=2, out_features=1):
        super(GeoffNet, self).__init__()

        # 输出层
        self.output_linear = nn.Linear(in_features=in_features, out_features=out_features)

    # 方法
    # => 前向传播: 让输入数据经过输入层 -> 隐藏层 -> 输出层; 逐层完成整合信息和加工信息的计算
    def forward(self, X):
        # 输入层 -> 输出层
        # 整合信息
        z_hat = self.output_linear(X)
        # 加工信息
        # output = nn.ReLU(z_hat)
        return z_hat

##### =====> 训练深度神经网络模型

In [94]:
# 实例化模型
net = GeoffNet()

# 选择损失函数
criterion = nn.MSELoss()

# 选择优化器
eta = 0.03
optimizer = optim.SGD(params=net.parameters(), lr=eta)

# 选择可视化工具
writer = SummaryWriter(logdir="reg_loss2")

# 搭建训练深度神经网络模型的流程
# =====> 将完整的数据集训练多遍
n_epochs = 3
for i_epoch in range(n_epochs):
    # =====> 将一个完整的数据集划分为若干个互不相交的子数据集进行训练
    for i_X, i_y in batched_dataset:
        # 1.前向传播
        i_y_hat = net.forward(i_X)
        # 2.计算损失
        loss = criterion(i_y_hat, i_y)
        # 3.利用损失函数的计算图, 反向传播
        loss.backward()
        # 4.利用反向传播的梯度信息, 使用优化器更新模型的参数
        optimizer.step()
        optimizer.zero_grad()
    # 在将一个完整的数据集训练一遍后, 计算以当前模型的参数进行学习, 得到的损失函数值.
    loss = criterion(net(X), y)
    writer.add_scalar("loss", loss, global_step=i_epoch)
    writer.add_graph(net, (X, ))

In [95]:
def fit(model, criterion, optimizer, X, y, n_epochs, batch_size):
    """搭建训练深度神经网络模型的流程
    @param: model, 需要进行训练的模型
    @param: criterion, 选择的损失函数
    @param: optimizer, 选择的优化器
    @param: n_epochs, 将完整的数据集训练多遍
    @param: batch_size, 将一个完整的数据集划分为若干个互不相交的子集, 一个子集的大小
    @param: X, 特征张量
    @param: y, 标签
    """
    # 使用TensorDataset类, 将特征张量和标签打包存储在元组中
    dataset = TensorDataset(X, y)
    # 将一个完整的数据集按照固定的子集大小划分为若干个子集
    batched_dataset = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last=False)
        
    # 搭建训练深度神经网络模型的流程
    # =====> 将完整的数据集训练多遍
    for i_epoch in range(n_epochs):
        # =====> 将一个完整的数据集划分为若干个互不相交的子数据集进行训练
        for i_X, i_y in batched_dataset:
            # 1.前向传播
            i_y_hat = net.forward(i_X)
            # 2.计算损失
            loss = criterion(i_y_hat, i_y)
            # 3.利用损失函数的计算图, 反向传播
            loss.backward()
            # 4.利用反向传播的梯度信息, 使用优化器更新模型的参数
            optimizer.step()
            optimizer.zero_grad()
        # 在将一个完整的数据集训练一遍后, 计算以当前模型的参数进行学习, 得到的损失函数值.
        loss = criterion(net(X), y)
        writer.add_scalar("loss", loss, global_step=i_epoch)
        writer.add_graph(net, (X, ))

In [88]:
net

GeoffNet(
  (output_linear): Linear(in_features=2, out_features=1, bias=True)
)

In [89]:
net.parameters()

<generator object Module.parameters at 0x1286eb450>

In [90]:
list(net.parameters())

[Parameter containing:
 tensor([[ 2.0000, -1.0001]], requires_grad=True),
 Parameter containing:
 tensor([0.9997], requires_grad=True)]

In [92]:
# 计算当前训练后的模型, 以当前的模型的参数完整的学习一遍数据集的损失函数的值
loss = criterion(net(X), y)
loss

tensor(9.4451e-05, grad_fn=<MseLossBackward0>)

In [93]:
writer.add_graph(net, (X, ))

In [99]:
# 加载数据集
torch.manual_seed(55)

X, y = tensorGenReg(degree=2)
X = X[:, :-1]

In [101]:
# 实例化模型
net = GeoffNet()
# 选择损失函数
criterion = nn.MSELoss()
# 选择优化器
optimizer = optim.SGD(params=net.parameters(), lr=0.03)

# 训练深度神经网络模型
n_epochs = 3
batch_size = 10
fit(model=net, criterion=criterion, optimizer=optimizer, X=X, y=y, n_epochs=n_epochs, batch_size=batch_size)

In [100]:
fit?

Signature: fit(model, criterion, optimizer, X, y, n_epochs, batch_size)
Docstring:
搭建训练深度神经网络模型的流程
@param: model, 需要进行训练的模型
@param: criterion, 选择的损失函数
@param: optimizer, 选择的优化器
@param: n_epochs, 将完整的数据集训练多遍
@param: batch_size, 将一个完整的数据集划分为若干个互不相交的子集, 一个子集的大小
@param: X, 特征张量
@param: y, 标签
File:      /var/folders/ht/7tjg01r945gch9nd9n7s7c3c0000gn/T/ipykernel_10257/3386227148.py
Type:      function

In [103]:
loss = criterion(net.forward(X), y)
loss

tensor(9.3560, grad_fn=<MseLossBackward0>)

In [107]:
torch.manual_seed(55)

# 加载数据集
X, y = tensorGenReg(delta=2)
X = X[:, :-1]

In [108]:
# 实例化模型
net = GeoffNet()
# 选择损失函数
criterion = nn.MSELoss()
# 选择优化器
optimizer = optim.SGD(params=net.parameters(), lr=0.03)

# 训练深度神经网络模型
n_epochs = 3
batch_size = 10
fit(model=net, criterion=criterion, optimizer=optimizer, X=X, y=y, n_epochs=n_epochs, batch_size=batch_size)

In [109]:
loss = criterion(net.forward(X), y)
loss

tensor(3.7781, grad_fn=<MseLossBackward0>)